# Upnote Markdown Manager Trial

## word splitting 

データフレームを作るところはjuliaで先に実行した。
ここではデータフレームの文書からsudachipyにて構文解析をする。

In [1]:
import polars as pl 
import sudachipy 
import os

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [66]:
import numpy as np

In [3]:
os.getcwd()

'd:\\develop\\upnote_md_mgr\\notebook'

In [4]:
dfm = pl.read_csv("../data/upnote_text.csv")

In [5]:
dfm.describe()

statistic,id,fpath,update,created,category,tags,contents
str,str,str,str,str,str,str,str
"""count""","""24673""","""24673""","""24673""","""24673""","""24673""","""10466""","""24673"""
"""null_count""","""0""","""0""","""0""","""0""","""0""","""14207""","""0"""
"""mean""",null,null,null,null,null,null,null
"""std""",null,null,null,null,null,null,null
"""min""","""000016e673d85d…","""- 名刺.md""","""2009-10-20T17:…","""2009-10-20T17:…","""0_業務""","""18の参加者|唄口リガチャー…",""" ""Trompeten, P…"
"""25%""",null,null,null,null,null,null,null
"""50%""",null,null,null,null,null,null,null
"""75%""",null,null,null,null,null,null,null
"""max""","""fff5a64a0679e1…","""ﾕcon.md""","""2024-03-06T22:…","""2024-03-06T22:…","""本""","""音響""",""" ﾕcon って何だ？ …"


HR872関連のデータのみに絞る。

In [6]:
dfm872 = dfm.filter(pl.col("tags") == "HR872")

In [7]:
dfm872

id,fpath,update,created,category,tags,contents
str,str,str,str,str,str,str
"""7d73c75ded161f…","""300Wモーター.md""","""2022-07-14T08:…","""2022-07-14T08:…","""仕事_Evernote""","""HR872""",""" 300Wモーター 金子ツ…"
"""55203fbd2731bd…","""HR 吹込管 口拡げ.md""","""2022-11-01T15:…","""2022-11-01T15:…","""仕事_Evernote""","""HR872""",""" HR 吹込管 口拡げ ど…"
"""8af8d31fb6fc3d…","""HR-672 吹込管.md""","""2022-07-26T09:…","""2022-07-26T09:…","""仕事_Evernote""","""HR872""",""" HR-672 吹込管 口…"
"""d19c050d6b9a7d…","""HR-672D ベルスクリュ…","""2022-06-13T08:…","""2022-06-13T08:…","""仕事_Evernote""","""HR872""",""" HR-672D ベルスクリ…"
"""cd3473406e90d8…","""HR-872 吹込管 2.m…","""2022-08-02T10:…","""2022-08-02T09:…","""仕事_Evernote""","""HR872""",""" HR-872 吹込管 三…"
…,…,…,…,…,…,…
"""b9a76854d102fc…","""管体+レバー支持台2 半田付…","""2022-10-28T16:…","""2022-10-28T16:…","""仕事_Evernote""","""HR872""",""" 管体+レバー支持台2 半田…"
"""cfe67cd7224278…","""管体組立 9.md""","""2022-10-26T20:…","""2022-10-26T20:…","""仕事_Evernote""","""HR872""",""" 管体組立 46班に行った…"
"""d5b76910f66938…","""設備手配表確認 2.md""","""2022-03-24T09:…","""2022-03-24T09:…","""仕事_Evernote""","""HR872""",""" 設備手配表確認 原さんか…"


In [8]:
dfm872[6,"contents"]

'\nHR-872 管体組立\n\n三宅氏の治具検図。\n\n駄目だ。まだまだだ。\n\n\n\n吹込管の入口テーパーが合ってないとか、ボルトのバランスが悪い、ブロック重そうなのにアルミでない、総形状受けやっている、\n\n668用の治具を修正するのに、昔の修正と同じ修正番号使っている、等々。\n\n\n\n\n\n\n'

sudachiによる構文解析をトライ。
ユーザー辞書を読み込めるように予め設定済。

In [9]:
# dict = sudachipy.Dictionary(config_path="../sudachi/sudachi.json")
dict = sudachipy.Dictionary()

In [10]:
tokenizer = dict.create()

In [11]:
tokens1 = tokenizer.tokenize(dfm872[6,"contents"])

In [12]:
for t in tokens1:
    if t.part_of_speech()[0] in ["名詞","形容詞","動詞"]:
        w = t.surface()
        n = t.normalized_form()
        if not w.isspace():
            if len(n) > 0:
                print(n, end=" ") 

HR 872 管体 組み立て 三宅 治具 検図 駄目 吹込 管 入り口 テーパー 合う ボルト バランス 悪い ブロック 重い アルミ 無い 形状 受ける 遣る 居る 668 治具 修正 為る 昔 修正 修正 番号 使う 居る 

キーワードを以下の手順で正規化されたスペース区切りの文に変換する。
- ユニコード文字列として正規化(全角英数は半角へ、半角カナなどは全角へ)
- 英数字は小文字化
- 図番に相当するas-j-99999の形のコードをasj99999型に変換
- sudachiで分かち書き
- 名詞・形容詞・動詞のみを抽出
- 正規化トークンに変換

In [13]:
import unicodedata
import re

一連の変換を関数にして、それを各行のデータに適用する。

In [24]:
def strtrans(s):
    tokens = [] # normalized token
    s1 = unicodedata.normalize("NFKC",s).lower()
    s2 = re.sub(r"([a-z]{2})-([djm])-([0-9]{4,5})", r"\1\2\3",s1) # replace drawing number
    tkn = tokenizer.tokenize(s2)
    for t in tkn:
        w = t.surface()
        if not w.isspace():
            if t.part_of_speech()[0] in ["名詞","形容詞","動詞"]:
                n = t.normalized_form()
                tokens.append(n)
    
    return (tokens)



In [25]:
def strtrans1(s):
    return(" ".join(strtrans(s)))

In [26]:
strtrans1(dfm872[1,"contents"])

'HR 吹込 管 口 広げる サンエイト 本当 自力 口 広げる 機 設置 為る よる HR 869 吹込 管 生産 ロット 入荷 為る 三宅 原 一緒 確認 以前 ロット 先端 口 広げる 為る 気配 無い こと 後 サンエイト 行く 確認 為る 設置 写真 取る'

データフレームに対してまとめて実行する

In [27]:
dfm872b = dfm872.with_columns([
    pl.col("contents").map_elements(strtrans).alias("tokens")
])

In [28]:
dfm872b[10,"tokens"]

'HR 672 吹込 管 口 広げる 治具 納品 納品書 佐藤 治具 資料室 files img 0013 jpg files img 0014 jpg'

全データフレームへの適用。
入力バイトが長すぎるというエラーが出るので、関数を改良する必要がある。

In [29]:
def strtrans2(s):
    tokens = []
    if len(s) > 2**14 -1:
        ss = s.split("\n")
        for sl in ss:
            tk =  strtrans(sl) 
            tokens = tokens + tk
    else:
        tokens = strtrans(s)

    return (" ".join(tokens))



In [30]:
dfm2 = dfm.with_columns([
    pl.col("contents").map_elements(strtrans2).alias("tokens")
])

## ginza, spacy

In [31]:
import spacy

In [32]:
nlp = spacy.load('ja_ginza')

In [33]:
doc = nlp(dfm872[6,"contents"])


In [34]:
for sent in doc.sents:
    for token in sent:
        print(token.text, end = "|")


|HR|-|872|管体|組立|

|三宅|氏|の|治具|検図|。|

|駄目|だ|。|まだまだ|だ|。|



|吹込|管|の|入口|テーパー|が|合っ|て|ない|と|か|、|ボルト|の|バランス|が|悪い|、|ブロック|重|そう|な|の|に|アルミ|で|ない|、|総|形状|受け|やっ|て|いる|、|

|668|用|の|治具|を|修正|する|の|に|、|昔|の|修正|と|同じ|修正|番号|使っ|て|いる|、|等々|。|






|

In [35]:
doc2 = nlp(dfm872[16,"contents"])


In [ ]:
doc2


HR672 吹込管 口拡げ

三宅氏の採寸結果を元に寸法調整。

曲げモーメントについて検討。A2017とA5052でヤング率に大きな違いがないので、円板の材料はA5052を採用することにした。

円板の曲げモーメントだが、応力と共に1/(1-v^2) (vはポワソン比)となるだけで、モーメントと応力との関係は直線梁と同じ関係(sigma = M/Z)になるので、

その曲がりにくさは、断面係数に依存するだけとなる。



3Dの詳細設計、ほぼ終了。既存機械との整合性もほぼ取れているのではないか?







In [37]:
doc.similarity(doc2)

0.9404588760527862

名詞句と形容詞句のみ取り出す。

In [38]:
doc3 = nlp("AS875EXの吹込管の大きな管引き芯金")

In [39]:
doc.similarity(doc3)

0.7587171220219052

特定の語を与え、それとの関連性の高い順にデータを並べ替えて表示する。

In [42]:
query_words = nlp(strtrans2("口拡げ治具"))

query_words

口 広げる 治具

In [45]:
dfm872c = dfm872b.with_columns([
    pl.col("tokens").map_elements(lambda x: query_words.similarity(nlp(x))).alias("similarity")
])

In [47]:
dfm872c.filter(pl.col("similarity") > 0.5).sort("similarity", descending=True)

id,fpath,update,created,category,tags,contents,tokens,similarity
str,str,str,str,str,str,str,str,f64
"""55203fbd2731bd…","""HR 吹込管 口拡げ.md""","""2022-11-01T15:…","""2022-11-01T15:…","""仕事_Evernote""","""HR872""",""" HR 吹込管 口拡げ ど…","""HR 吹込 管 口 広げる …",0.676491
"""9c2fce7cdaeea0…","""HR872 吹込管 7.md…","""2023-02-16T14:…","""2023-02-16T13:…","""仕事_Evernote""","""HR872""",""" HR872 吹込管 サン…","""HR 872 吹込 管 サン…",0.630001
"""d5d281cce60370…","""HR872 吹込管 3.md…","""2023-03-20T08:…","""2023-03-20T08:…","""仕事_Evernote""","""HR872""",""" HR872 吹込管 細端…","""HR 872 吹込 管 細い…",0.583208
"""f4ba5f42a993fa…","""HR672 吹込管 口拡げ機…","""2022-07-28T14:…","""2022-07-28T14:…","""仕事_Evernote""","""HR872""",""" HR672 吹込管 口拡げ…","""HR 672 吹込 管 口 …",0.581679
"""d017b2efd91cae…","""吹込管口拡げ.md""","""2022-10-14T18:…","""2022-10-14T13:…","""仕事_Evernote""","""HR872""",""" 吹込管口拡げ サンエイト…","""吹込 管口 広げる サンエイ…",0.57921
…,…,…,…,…,…,…,…,…
"""aacf582429189d…","""HR672 吹込管口拡げ.m…","""2022-11-18T21:…","""2022-11-18T21:…","""仕事_Evernote""","""HR872""",""" HR672 吹込管口拡げ …","""HR 672 吹込 管口 広…",0.507849
"""8f9f44ad0dae04…","""HR吹込管口拡げ.md""","""2022-09-22T21:…","""2022-09-22T15:…","""仕事_Evernote""","""HR872""",""" HR吹込管口拡げ 原さん…","""HR 吹込 管口 広げる 原…",0.507663
"""d58897f5a8bae7…","""HR872 レバー4.md""","""2023-01-10T17:…","""2023-01-10T17:…","""仕事_Evernote""","""HR872""",""" HR872 レバー4 三…","""HR 872 レバー 4 三…",0.507454


In [48]:
dfm872.shape

(135, 7)

nlpを行うときも、入力文字数が多すぎるとエラーとなるので、分割して計算する。
一致度は複数の一致度の平均にする。

In [67]:
def calc_similarity(s):
    MAX_LENGTH = 2**14 -1
    chunks = []
    start = 0

    if len(s) > MAX_LENGTH:
        while start < len(s):
            end = start + MAX_LENGTH
            if end < len(s):

                lsp = s.rfind(r"\s", start, end)
            ch = s[start:lsp]
            chunks.append(ch)
            start = lsp + 1

        slist = np.array([query_words.similarity(nlp(ch)) for ch in chunks])
        smlty = np.mean(slist)

    else:
        smlty = query_words.similarity(nlp(s))

    return(smlty)
        


In [68]:
calc_similarity(dfm2[10,"tokens"])

0.39730599184023413

次のコードはメモリーエラーになる。

In [69]:
# dfm3 = dfm2.with_columns([
#     pl.col("tokens").map_elements(lambda x: calc_similarity(x)).alias("similarity")
# ])

PanicException: python function failed MemoryError: 

PanicException: python function failed MemoryError: 

## test

In [87]:
re.sub(r"([a-z]{2})-([djm])-([0-9]+)", r"\1\2\3","as-j-12345")

'asj12345'

In [18]:
len(bytes("日本語のテスト".encode("UTF-8")))

21

In [19]:
len("日本語のテスト")

7

In [20]:
49149/3

16383.0

In [21]:
import numpy as np

In [22]:
np.log2(49149/3)

13.99991194219542

In [23]:
2**14

16384

In [64]:
MAX_LENGTH = 2400

s1 = "aset asetas as aas a  ahoinhaso  a ahoin. asetn .asetae asas"
last_space_pos = s1.rfind(' ', 0, MAX_LENGTH)

In [65]:
s1[0:last_space_pos]

'aset asetas as aas a  ahoinhaso  a ahoin. asetn .asetae'

In [57]:
s1[0:ans.endpos-1]

'aset asetas as aas a  ahoinhaso  a ahoin. asetn .asetae asa'